In [1]:
from iwi131_utils import iwi131_utils
import logging
import os
import pandas as pd 

pjoin = os.path.join

logging.basicConfig(level=getattr(logging, 'INFO'))
logger = logging.getLogger('get_notas_csj')

In [2]:
# VARIABLES DE ENTORNO

# Contests de hackerrank
contests =['https://www.hackerrank.com/01-apoquindo-sj','https://www.hackerrank.com/09-manquehue',
'https://www.hackerrank.com/04-el-plomo','https://www.hackerrank.com/02-calan',
'https://www.hackerrank.com/10-torrecillas','https://www.hackerrank.com/12-provincia-sj',
'https://www.hackerrank.com/06-la-cruz','https://www.hackerrank.com/08-leonera',
'https://www.hackerrank.com/11-pintor']

# Plantilla de SIGA para las listas
siga_lista='Curso_IWI131_{0}_2019-1.xls'

# Paralelos
paralelos=range(200,215)

# Directorio de listas
dir_listas = pjoin('/home/jovyan/listas_2019s1_csj/')

# Directorio de notas
dir_notas = pjoin('/home/jovyan/notas_lab1_2019s1_csj/')

In [ ]:
# Diccionario de DFs por paralelo
dic_df_par={}

# Obtencion de los alumnos desde las listas del SIGA
logger.debug('++ Obteniendo datos de las listas de SIGA...')
for par in paralelos:
    curso=pjoin(dir_listas, siga_lista.format(par))
    dic_df_par[par]=iwi131_utils.estudiantes_x_paralelo_siga(curso)
logger.debug('++ OK')

In [ ]:
# Lista con los alumnos y notas desde los leaderboard
lista_alumnos_nota=[]

logger.debug('++ Obteniendo notas desde todos los contests...')
#Obtencion de los usuarios y notas de los leaderboard
for con in contests:
    scr = iwi131_utils.Scraper(con+'/leaderboard')
    lista_alumnos_nota+=[(sansano_hack.username,sansano_hack.nota) for sansano_hack in scr.scrape()]
logger.debug('++ OK')

# DF con todas las notas y los usuarios
df_alumnos_nota=pd.DataFrame(lista_alumnos_nota, columns = ['username', 'nota']) 

/opt/conda/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '
INFO:iwi131_utils:Loading leaderboard from url https://www.hackerrank.com/01-apoquindo-sj/leaderboard
INFO:iwi131_utils:Page 1 loaded.
INFO:iwi131_utils:Page 2 loaded.
INFO:iwi131_utils:Page 3 loaded.
INFO:iwi131_utils:Page 4 loaded.
INFO:iwi131_utils:Page 5 loaded.
INFO:iwi131_utils:Page 6 loaded.
INFO:iwi131_utils:Leaderboard details not found -- Done loading
INFO:iwi131_utils:Loading leaderboard from url https://www.hackerrank.com/09-manquehue/leaderboard
INFO:iwi131_utils:Page 1 loaded.
INFO:iwi131_utils:Page 2 loaded.
INFO:iwi131_utils:Page 3 loaded.


In [ ]:
# Diccionario llave paralelo y valor un df con los roles,user y nota de cada alumno
match_alumnos_notas={}

# Merge entre todas las notas y los alumnos por paralelo
logger.debug('++ Empaquetando las notas por paralelo...')
for paralelo in paralelos:
    match_alumnos_notas[paralelo]=pd.merge(df_alumnos_nota,dic_df_par[paralelo], on="username")
logger.debug('++ OK')    

In [ ]:
# Guarda las notas de los paralelos
logger.debug('++ Guardando notas en CSV por paralelo...')
for paralelo,df in match_alumnos_notas.items():
    df.to_csv(pjoin(dir_notas,'NOTAS_LAB1_{0}'.format(paralelo)))
logger.debug('++ OK')

In [ ]:
logger.debug('++ Obteniendo alumnos no identificados...')
# DF con los alumnos sin identificacion en el match por paralelo
alumnos_sin_identificar=pd.concat([pd.concat(match_alumnos_notas.values()), df_alumnos_nota]).drop_duplicates(subset='username', keep=False)

# Guarda en un CVS las notas de los alumnos sin identificar
alumnos_sin_identificar.to_csv(pjoin(dir_notas,'NOTAS_LAB1_NN'.format(paralelo)))
logger.debug('++ OK')

In [ ]:
logger.debug('+++ Notas listas !')